# Global and Local Scoping

### When python encounters a function definition at compile-time
1. It will `scan` for any labels that have values `assigned` to them (anywhere in the function). If the balen has not been specified as `global`, then it will be local.
2. Variables that are referenced but `not assigned` a value anywhere in the function will `not be local` and python will, at `run-time`, look for them in `enclosing` scopes.

In [8]:
a = 10

def my_func():
    print(a)
    
my_func()

print(a)

10
10


In [9]:
a = 10

def my_func():
    a = 'hello'
    print(a)

my_func()
print(a)

hello
10


In [10]:
a = 10

def my_func():
    global a
    a = 'hello'
    print(a)

my_func()
print(a)

hello
hello


In [13]:
# at compile-time python determined a to be local, 
# at run-time python looks for a local a which has not been defined.

a = 10

def my_func():
    print(f'global a: {a}')
    a = 'hello'
    print(a)

my_func()
print(a)

UnboundLocalError: cannot access local variable 'a' where it is not associated with a value

In [14]:
print(x)

NameError: name 'x' is not defined

In [15]:
for i in range(10):
    x = i ** 2

In [19]:
# variables in python are not block scoped
i, x

(9, 81)

# Nonlocal Scopes

In [20]:
def outer_func():
    x = 'hello'  # enclosing scope
    def inner_func():
        print(x)
    inner_func()

In [21]:
outer_func()

hello


In [23]:
def outer_func():
    x = 'hello'
    def inner1():
        def inner2():
            print(x)
        inner2()
    inner1()

outer_func()

hello


In [26]:
def outer_func():
    x = 'hello'
    def inner():
        x = 'python'
        print(f'inner x: {x}')
    inner()
    print(f'outer x: {x}')

outer_func()

inner x: python
outer x: hello


In [31]:
def outer_func():
    x = 'hello'
    def inner():
        nonlocal x
        x = 'python'
        print(f'inner x: {x}')
    print(f'outer(before) x: {x}')
    inner()
    print(f'outer(after) x: {x}')

outer_func()

outer(before) x: hello
inner x: python
outer(after) x: python


# Closures

In [44]:
def outer():
    x = 'python'

    def inner():
        print(f'{x}')
    
    inner()

outer()

python


In [35]:
def outer():
    x = 'python'

    def inner():
        print(f'{x}')
    
    return inner

fn = outer()

In [37]:
type(fn)

function

# Python creates a cell. outer x points and inner x both point to the cell which points to the string 'python'

In [48]:
fn.__code__.co_freevars
# gives free variables

('x',)

In [49]:
fn.__closure__

(<cell at 0x108025ba0: str object at 0x100b72400>,)

In [50]:
hex(id(x))

'0x100b72400'

In [46]:
fn.__closure__ = 'brk'

AttributeError: readonly attribute

# closure applications

In [54]:
class Averager:
    def __init__(self):
        self.numbers = []

    def add(self, number):
        self.numbers.append(number)
        return sum(self.numbers)/len(self.numbers)

In [56]:
a = Averager()
print(a.add(10))
print(a.add(20))
print(a.add(30))

10.0
15.0
20.0


In [57]:
b = Averager()
print(b.add(10))
print(b.add(20))
print(b.add(30))

10.0
15.0
20.0


In [58]:
def averager():
    l = []
    def inner(number):
        l.append(number)
        return sum(l)/len(l)
    return inner
        

In [62]:
a = averager()
print(a(10))
print(a(20))
print(a(30))

10.0
15.0
20.0


In [63]:
b = averager()
print(b(10))
print(b(20))
print(b(30))

10.0
15.0
20.0


## a and b have cells pointing to different instance of list `l`

In [66]:
a.__code__.co_freevars, a.__closure__

(('l',), (<cell at 0x1080ae890: list object at 0x10846d640>,))

In [67]:
b.__code__.co_freevars, b.__closure__

(('l',), (<cell at 0x1080aea40: list object at 0x108425200>,))

In [77]:
# a better way - lesser time complexity by avoiding 2 iterations

def averager():
    total = 0
    count = 0 
    def inner(number):
        total += number
        count += 1
        return total / count
    return inner

In [78]:
# total variable has been assigned so python thinks of it as local variable
# and evaluates total to right of = sign and finds no local var!

a = averager()
print(a(10))
print(a(20))
print(a(30))

UnboundLocalError: cannot access local variable 'total' where it is not associated with a value

In [79]:
def averager():
    total = 0
    count = 0 
    def inner(number):
        nonlocal total
        nonlocal count
        total += number
        count += 1
        return total / count
    return inner

In [80]:
a = averager()
print(a(10))
print(a(20))
print(a(30))

10.0
15.0
20.0


In [82]:
a.__code__.co_freevars

('count', 'total')

In [83]:
a.__closure__

(<cell at 0x1080b8eb0: int object at 0x1013a6100>,
 <cell at 0x1080b9780: int object at 0x1013a6820>)